In [1]:
!pip install geopy requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.4 MB/s eta 0:00:00


In [2]:
import requests
from geopy.distance import great_circle

In [3]:
url = "https://raw.githubusercontent.com/cristiroma/countries/main/data/countries.json"
response = requests.get(url)
response.raise_for_status()
geodata = response.json()
geodata[0], len(geodata)

({'id': '1',
  'enabled': True,
  'code3l': 'AFG',
  'code2l': 'AF',
  'name': 'Afghanistan',
  'name_official': 'Islamic Republic of Afghanistan',
  'center': {'latitude': '33.98299275',
   'longitude': '66.39159363',
   'zoom': '6'},
  'names': {'ar': {'name': 'أفغانستان',
    'name_official': 'جمهورية أفغانستان الإسلامية '},
   'es': {'name': 'Afganistán',
    'name_official': 'la República Islámica del Afganistán'},
   'fr': {'name': 'Afghanistan',
    'name_official': "la République islamique d'Afghanistan"},
   'it': {'name': 'Afghanistan',
    'name_official': 'Repubblica Islamica di Afghanistan  '},
   'ru': {'name': 'Афганистан',
    'name_official': 'Исламская Республика Афганистан'},
   'zh': {'name': '阿富汗', 'name_official': '阿富汗伊斯兰共和国'}}},
 250)

In [4]:
url = "https://github.com/annexare/Countries/raw/main/dist/countries.min.json"
response = requests.get(url)
response.raise_for_status()
langdata = response.json()
langdata['MX'], len(geodata)

({'name': 'Mexico',
  'native': 'México',
  'phone': [52],
  'continent': 'NA',
  'capital': 'Mexico City',
  'currency': ['MXN'],
  'languages': ['es']},
 250)

In [5]:
data = []
for geo in geodata:
    if not geo['code2l'] in langdata:
        continue
    lang = langdata[geo['code2l']]
    row = {
        "code": geo['code2l'],
        "name": lang['name'],
        "lang": lang['languages'][0] if len(lang['languages']) > 0 else 'en',
        "currency": lang['currency'][0].lower() if len(lang['currency']) > 0 else 'usd',
        "tel": lang['phone'][0],
        "lon": geo['center']['longitude'],
        "lat": geo['center']['latitude'],
    }
    data.append(row)

In [6]:
data[0]

{'code': 'AF',
 'name': 'Afghanistan',
 'lang': 'ps',
 'currency': 'afn',
 'tel': 93,
 'lon': '66.39159363',
 'lat': '33.98299275'}

In [7]:
server_locations = {
    'sga.domcloud.co': (1.3521, 103.8198),  # Singapore
    'nyc.domcloud.co': (40.7128, -74.0060), # New York
    'ams.domcloud.co': (52.3676, 4.9041),   # Amsterdam
    'tyo.domcloud.co': (35.6895, 139.6917), # Tokyo
    'blr.domcloud.co': (12.9716, 77.5946),  # Bangalore
    'sao.domcloud.co': (-23.5505, -46.6333) # Sao Paulo
}

currencies = ['usd', 'idr']

languages = ['ar', 'de', 'en', 'id', 'fr', 'hi', 'es', 'it', 'ja', 'ko', 'nl', 'pt', 'ru', 'zh']

# Function to find the nearest server for a given country
def find_nearest_server(country_coords):
    min_distance = float('inf')
    nearest_server = None
    for server, server_coords in server_locations.items():
        distance = great_circle(country_coords, server_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_server = server
    return nearest_server

newdata = data.copy()
for row in newdata:
    row['currency'] = row['currency'] if row['currency'] in currencies else 'usd'
    row['lang'] = row['lang'] if row['lang'] in languages else 'en'
    row['server'] = find_nearest_server(( row['lat'],  row['lon']))
    del(row['lon'])
    del(row['lat'])



In [8]:
newdata[42]

{'code': 'CF',
 'name': 'Central African Republic',
 'lang': 'fr',
 'currency': 'usd',
 'tel': 236,
 'server': 'ams.domcloud.co'}